# 0. Data

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [15]:
# first 21 columns are input variables and last 7 columns are output variables

train_data = pd.read_csv('./data/SARCOSTst.csv', header=None)
test_data = pd.read_csv('./data/SARCOSTrn.csv', header=None)

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X_rem,y_rem, test_size=0.5)



ValueError: test_size=1.0 should be either positive and smaller than the number of samples 7897 or a float in the (0, 1) range

# 1. Nonlinear GP regression
- Gaussian kernel

# 2.Linear GP regression 
- Standard dot product kernel
    - k(a,b) = a^\top b